We'll start by plotting a simple map. 

In [ ]:
# Import modules 
import folium
import geopandas as gpd
import json
from branca.element import Template, MacroElement

# Define function to plot initial map
def init_map():
  m = folium.Map(
    location=[51.9, 4.3],
    zoom_start=11.4,
    zoom_control=False,
    scrollWheelZoom=False,
    dragging=False
  )

  return m

map = init_map()
map.save('map.html')
map

Now let's plot routes on it. Before we can do this, we will need to convert our routes to a suitable input format. 

In [ ]:
def PlotRoutes(date, type):
    # Get information
    data_input = json.load(open('solutions/main_analysis/solution_{date}_{type}.json'.format(date=date, type=type)))
    routes = data_input['Ordered_AIS']

    # Initialize map & colors
    colors = ['blue', 'red','green','yellow'] 
    m = init_map()
    depots = [[51.96249648967369, 4.137689887873459],[51.89740273617377, 4.460817447562738]] # DP locations
    supplypoints = [[51.889365439900516, 4.313357332833194]] # SP locations

    # Adding the routes
    for i, (boat, route) in enumerate(routes.items()):
        polyline = folium.PolyLine(locations=route, color=colors[i], weight=2.5, opacity=1)
        polyline.add_child(folium.Tooltip(boat))
        m.add_child(polyline)

        # Add markers for each location
        for location in route:
            if location in depots:
                folium.Marker(location, icon=folium.Icon(color='blue', icon='parking', prefix='fa')).add_to(m)
            elif location in supplypoints:
                folium.Marker(location, icon=folium.Icon(color='red', icon='gas-pump', prefix='fa')).add_to(m)
            else:
                #folium.Marker(location).add_to(m)
                pass

    # Add HTML code
    template = """
    {% macro html(this, kwargs) %}
    <div style="
        position: fixed; 
        bottom: 580px;
        left: 270px;
        width: 120px;
        height: 90px; 
        z-index:9999;
        font-size:14px;
        ">
        <p><a style="color:blue;">&#9679;</a>&nbsp;Watergeus</p>
        <p><a style="color:red;">&#9679;</a>&nbsp;Waterbuffel</p>
        <p><a style="color:green;">&#9679;</a>&nbsp;Waterval</p>
        <p><a style="color:yellow;">&#9679;</a>&nbsp;Waterman</p>
    </div>
    <div style="
        position: fixed; 
        bottom: 550px;
        left: 250px;
        width: 120px;
        height: 130px; 
        border:2px solid grey; 
        z-index:9998;
        font-size:14px;
        background-color:white;
        opacity: 0.7;
        ">
    </div>
    {% endmacro %}
    """

    macro = MacroElement()
    macro._template = Template(template)

    m.get_root().add_child(macro)

    # Save the map to an HTML file
    m.save(outfile='plots/solplots/html/map_{date}_{type}.html'.format(date=date,type=type))
    
    return(m)

In [ ]:
# Define some sample routes
dates = ['2022-01-27', '2022-03-08','2022-03-09','2022-05-03','2022-05-19','2022-06-29','2022-07-06','2022-07-26','2022-07-28','2022-09-08','2022-09-29','2022-09-30','2022-10-04','2022-10-14','2022-10-21']
type = "regular"

for i in dates:
    PlotRoutes(i, type)